<a href="https://colab.research.google.com/github/SonOfADev/memes_vs_notes/blob/master/memes_vs_notes_refined.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch 
import numpy as np
import PIL
from PIL import Image
import os
 
folder = "/content/drive/My Drive/Memes_vs_Notes/"
classes = ["Memes", "Notes"]

In [2]:
from sklearn.utils import shuffle
import cv2
def load_data(folder, classes, IMG_HEIGHT, IMG_WIDTH):
    data = []
    data_dnn = []
    labels = []
    label = -1
    for y in classes:
        label += 1
        path = folder + str(y)+"/"
        for x in os.listdir(path):
            img = cv2.imread(path + str(x))
            img = cv2.resize(img,(IMG_WIDTH, IMG_HEIGHT))
            data.append(img)       
            labels.append(label)
    dataset = np.array([np.array(x) for x in data])
    dataset, labels = shuffle (dataset, labels, random_state=0) 
    dataset = torch.Tensor(dataset)
    labels = torch.Tensor(labels)  
     
    return dataset, labels

In [3]:
IMG_HEIGHT = 100
IMG_WIDTH git remote add origin https://github.com/SonOfADev/memes_vs_notes.git= 100
data, labels = load_data(folder, classes, IMG_HEIGHT, IMG_WIDTH)

In [5]:
device =  torch.device("cpu")
data = data/255

data.shape, labels.shape

(torch.Size([1600, 100, 100, 3]), torch.Size([1600]))

In [6]:
import torch.nn as nn
class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        # self.layer5 = nn.Sequential(
        #     nn.Conv2d(256, 512, kernel_size=5, stride=1, padding=2),
        #     nn.ReLU(),
        #     nn.MaxPool2d(kernel_size=2, stride=2))
        # self.layer6 = nn.Sequential(
        #     nn.Conv2d(512, 258, kernel_size=1, stride=1),
        #     nn.ReLU()
        # )
        #self.drop_out = nn.Dropout()
        self.fc1 = nn.Sequential(
            nn.Linear(6 *6 * 256,1000),
            nn.ReLU())
        self.fc2 = nn.Linear(1000, 1)

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        # out = self.layer5(out)
        # out = self.layer6(out)
        out = out.reshape(out.size(0),-1)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [7]:
model = ConvNet()
model.to(device)
learning_rate = 0.001
# Loss and optimizer
criterion =torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
data_resized = data.reshape(-1,3,100,100)
#data_resized = torch.Tensor(data_resized)
train_data = data_resized[:1000]
train_labels = labels[:1000]
valid_data = data_resized[1000:1300]
valid_labels = labels[1000:1300]
test_data = data_resized[1300:1600]
test_labels = labels[1300:1600]
np.shape(train_data)

train_data = train_data.to(device)
train_labels = train_labels.to(device)

valid_data = valid_data.to(device)
valid_lables = valid_labels.to(device)

test_data = test_data.to(device)
test_labels = test_labels.to(device)
train_data.shape

torch.Size([1000, 3, 100, 100])

In [9]:
#train_labels = train_labels.type(torch.LongTensor) 
train_labels = train_labels.unsqueeze(-1)
valid_labels = valid_labels.unsqueeze(-1)
test_labels = test_labels.unsqueeze(-1)
train_labels.type(), train_labels.shape

('torch.FloatTensor', torch.Size([1000, 1]))

In [12]:
import torch.nn.functional as F
num_epochs = 10
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    total = 0
    correct = 0

    outputs = model(train_data)
    loss = criterion(outputs, train_labels)
    loss_list.append(loss.item())

    # Backprop and perform Adam optimisation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Track the accuracy
        
    with torch.no_grad():
        #outputs = net(images)
        outputs_valid = model(valid_data)
        valid_loss = criterion(outputs_valid, valid_labels)

        predicted_valid = (torch.sigmoid(outputs_valid.data)>0.5)

        predicted = (F.sigmoid(outputs.data)>0.5)

        total = train_labels.size(0)
        correct = (predicted == train_labels).sum().item()

        total_valid = valid_labels.size(0)
        correct_valid = (predicted_valid == valid_labels).sum().item()

    print("training accuracy %d %%" %(100*correct/total ))
    print('validation Accuracy: %d %%' % (100 * correct_valid / total_valid))
    print(f"train loss = {loss}")
    print(f"validation loss = {valid_loss}")

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1625: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


training accuracy 90 %
validation Accuracy: 92 %
train loss = 0.2628377377986908
validation loss = 0.258310467004776
training accuracy 93 %
validation Accuracy: 92 %
train loss = 0.18582525849342346
validation loss = 0.2797485589981079
training accuracy 93 %
validation Accuracy: 92 %
train loss = 0.20515060424804688
validation loss = 0.26262524724006653
training accuracy 93 %
validation Accuracy: 93 %
train loss = 0.18584175407886505
validation loss = 0.2434297502040863
training accuracy 95 %
validation Accuracy: 92 %
train loss = 0.14151953160762787
validation loss = 0.27091842889785767
training accuracy 94 %
validation Accuracy: 92 %
train loss = 0.16468903422355652
validation loss = 0.25354695320129395
training accuracy 95 %
validation Accuracy: 94 %
train loss = 0.14452633261680603
validation loss = 0.20631812512874603
training accuracy 96 %
validation Accuracy: 95 %
train loss = 0.1184215173125267
validation loss = 0.21137364208698273
training accuracy 95 %
validation Accuracy: 95

In [ ]:
MODEL_STORE_PATH = "/content/drive/"
torch.save(model.state_dict(), MODEL_STORE_PATH + 'conv_net_model.ckpt')
